# Emotion Recognition of Audios in Snowflake using Snowpark Python, OpenAI and NRCLex

_For comments and feedback, please reach out to [Divyansh](https://www.linkedin.com/in/divyanshsaxena/)_.


In [1]:
# Snowpark
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf
from snowflake.snowpark.version import VERSION

# Misc
import logging 
import config
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

### Establish Secure Connection to Snowflake

Using the Snowpark API, it’s quick and easy to establish a secure connection between Snowflake and Notebook.

 *Connection options: Username/Password, MFA, OAuth, Okta, SSO*

In [2]:
# Create Snowflake Session object
session = Session.builder.configs(config.sf_conn_config).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(snowflake_environment[0][1]))
print('Database                    : {}'.format(snowflake_environment[0][2]))
print('Schema                      : {}'.format(snowflake_environment[0][3]))
print('Warehouse                   : {}'.format(snowflake_environment[0][5]))
print('Snowflake version           : {}'.format(snowflake_environment[0][4]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : DIVYANSH
Role                        : SYSADMIN
Database                    : OPEN_AI_DB
Schema                      : EMOTION_RECOGNITION
Warehouse                   : COMPUTE_WH
Snowflake version           : 7.11.6
Snowpark for Python version : 1.1.0


### Snowpark Python User-Defined Function (UDF) for Emotion Recognition of Audio Files

Now to use custom python libraries which are not available on snowpark anaconda channel, we have to zip those libraries and need to upload those packages on snowflake.

This can be done using the below commands:

```
pip install snowflake-cli-labs

snow package create <package-name>
```
To know more on how to use custom packages on snowflake which are not available on Snowflake Anaconda Channel, refer to the [Medium Article by Jeff Hollan](https://medium.com/snowflake/using-other-python-packages-in-snowpark-a6fd75e4b23a)

*NOTE: Scalar UDFs operate on a single row / set of data points and are great for online inference in real-time. And this UDF is called from [Snowpark_NRCLex_Streamlit_OpenAI_Audio_rec.py](Snowpark_NRCLex_Streamlit_OpenAI_Audio_rec.py) Streamlit apps.*

TIP: For more information on Snowpark Python User-Defined Functions, refer to the [docs](https://docs.snowflake.com/en/developer-guide/snowpark/python/creating-udfs.html).

In [3]:
#Uploading the Custom Python Packages on the Snowflake
session.file.put('NRCLex.zip','@packages',overwrite=True,auto_compress=False)

[PutResult(source='NRCLex.zip', target='NRCLex.zip', source_size=56518948, target_size=56518960, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [4]:
session.clear_packages()
session.clear_imports()

# Add Python packages from Snowflke Anaconda channel OR Custom Packages
session.add_import('@packages/NRCLex.zip')
session.add_packages('click', 'colorama', 'joblib', 'nltk', 'regex', 'tqdm')

In [5]:

@udf(name='sp_emotion_recognition',session=session,replace=True,is_permanent=True,stage_location='@raw_data_set/procedures/')
def sp_emotion_recognition(text_data: str) -> str:
  import nltk
  nltk.download('punkt')
  from nrclex import NRCLex
  emotion = NRCLex(text_data)
  return f"{emotion.top_emotions}"